In [ ]:
import itertools as it
from collections import Counter, defaultdict
from functools import partial
from glob import glob
from pathlib import Path

import gfapy
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
pl.enable_string_cache()

# Functions

In [ ]:
def concat_glob(filename):
    return pl.concat([pl.scan_ipc(f) for f in glob(filename)], how="diagonal")

In [ ]:
def label_columns(cols, func=None):
    expr = None
    for col in cols:
        if expr is None:
            expr = pl.when(pl.col(col).is_not_null())
        else:
            expr = expr.when(pl.col(col).is_not_null())
        if func is not None:
            lit = func(col)
        else:
            lit = col
        expr = expr.then(pl.lit(lit))
    return expr

In [ ]:
def load_sequencing(filename, filter=True):
    df = concat_glob(filename)
    df = df.with_columns(
        dup=pl.col("name").is_duplicated(),
        primary_alignment=pl.col("name").is_first_distinct(),
        e2e=pl.col("variants_path")
        .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
        .list.len()
        == 2,
        bc_e2e=pl.col("variants_path")
        .list.set_intersection(
            ["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"]
        )
        .list.len()
        == 2,
    )
    if filter:
        df = df.filter(pl.col("primary_alignment"), pl.col("e2e"))
    df = processing.compute_divergences(df, [f"BC:bit{idx}" for idx in range(30)])
    return df

In [ ]:
def path_to_barcode_string(path_col, bits=list(range(30))):
    if isinstance(path_col, str):
        path_col = pl.col(path_col)
    return pl.concat_str(
        [
            pl.when(
                path_col.list.contains(f">BC:bit{bit}=1").or_(
                    path_col.list.contains(f"<BC:bit{bit}=1")
                )
            )
            .then(pl.lit("1"))
            .otherwise(pl.lit("0"))
            for bit in bits
        ]
    )

# 240703_pLIB476_bottlenecked

In [ ]:
%%time
df = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/references/pLIB476jqs.gfa"
)

In [ ]:
df = processing.compute_divergences(
    df, set([s.split("=")[0] for s in gfa.segment_names])
)

In [ ]:
df["max_divergence"].value_counts().sort("max_divergence")

In [ ]:
plt.hist(df4["max_divergence"], bins=100);

In [ ]:
df3.filter(pl.col("grouping_depth") < 10)

In [ ]:
df3.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df4.filter(pl.col("max_divergence") > 0.05).sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
%%time
df3.select(processing.categorical_list_hash(pl.col("grouping_path")).unique())

In [ ]:
df3["grouping_depth"].value_counts().sort("grouping_depth").with_columns(
    cum=pl.col("count").cum_sum(reverse=True)
).to_pandas().hvplot.step("grouping_depth", "cum", logy=True)

In [ ]:
plt.hist(df3["grouping_depth"], bins=1000, log=True);

## Burden

In [ ]:
df["SD2_variant|seq"].value_counts(sort=True).with_columns(
    frac=pl.col("count") / pl.col("count").sum(),
).to_pandas()["count"].hvplot.step(logy=True)

In [ ]:
df.filter(pl.len().over("SD2_variant|seq") > 1).group_by("SD2_variant|seq").agg(
    "grouping_depth"
).sort(pl.col("grouping_depth").list.mean()).with_columns(
    rbs_id=pl.int_range(pl.len())
).select(
    "rbs_id", "grouping_depth"
).explode(
    "grouping_depth"
).to_pandas().hvplot.scatter(
    "rbs_id", "grouping_depth", alpha=0.2, s=2
)

# Joins

##  240703_pLIB476_bottlenecked with 240612_pLIB476_bottlenecked

In [ ]:
%%time
df_240703b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()
df_240610b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
%%time
cols = ["grouping_path", "variants_path", "grouping_depth"]
df_joined = df_240703b.select(cols).join(
    df_240610b.select(cols),
    on=processing.categorical_list_hash(pl.col("grouping_path")),
    how="full",
    suffix="_240610b",
)

In [ ]:
df_joined.group_by(
    pl.col("grouping_path").is_not_null(), pl.col("grouping_path_240610b").is_not_null()
).agg(pl.len()).rename(
    {
        "grouping_path": "240703_bottlenecked",
        "grouping_path_240610b": "240612_minion_bottlenecked",
    }
)

In [ ]:
df_240703b.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df_240610b.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df_joined.select("grouping_depth", "grouping_depth_240610b").to_pandas().hvplot.scatter(
    "grouping_depth", "grouping_depth_240610b"
)

##  240703_pLIB476_bottlenecked with 240510_pLIB476

In [ ]:
%%time
df_240703b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()
df_240513 = load_sequencing(
    "/home/jqs1/scratch/sequencing/240513_pLIB473_476/20240513_1645_2C_PAW46239_b49d575f/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
%%time
cols = ["grouping_path", "variants_path", "grouping_depth"]
df_joined = df_240703b.select(cols).join(
    df_240610b.select(cols),
    on=processing.categorical_list_hash(pl.col("grouping_path")),
    how="full",
    suffix="_240610b",
)

In [ ]:
df_joined.group_by(
    pl.col("grouping_path").is_not_null(), pl.col("grouping_path_240610b").is_not_null()
).agg(pl.len()).rename(
    {
        "grouping_path": "240703_bottlenecked",
        "grouping_path_240610b": "240612_minion_bottlenecked",
    }
)

## J23100

In [ ]:
align.pairwise_align(
    "GGAGTTTACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC", "GGAGTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC"
)

In [ ]:
x = df.filter(pl.col("promoter|seq") == "GGAGTTTACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC")

In [ ]:
x[0, "cg"]

In [ ]:
x[0, "realign_cg"]

In [ ]:
idx = 0
path = x[idx, "variants_path"]
seq = x[idx, "consensus_seq"]

In [ ]:
name_to_seq = sgfa.gfa_name_mapping(gfa)
ref_seq = sgfa.assemble_seq_from_path(name_to_seq, path)

In [ ]:
seq

In [ ]:
processing.pairwise_align(
    seq,
    ref_seq,
    **{"cigar_as_string": True, "method": "parasail", "degenerate": True},
)

In [ ]:
processing.pairwise_align(
    seq,
    ref_seq,
    **{"cigar_as_string": True, "method": "parasail", "degenerate": False},
)

In [ ]:
x.columns

In [ ]:
y = processing.cut_cigar_df(
    x,
    gfa,
    path_column="variants_path",
    cigar_column="cg",
    sequence_column="consensus_seq",
    # phred_column="consensus_phred",
    query_start_column="query_start",
    query_end_column="query_end",
    query_length_column="query_length",
    path_start_column="path_start",
    path_end_column="path_end",
    struct_name="extract_segments2",
    keep_full=True,
)

In [ ]:
y2 = processing.cut_cigar_df(
    x,
    gfa,
    path_column="variants_path",
    cigar_column="realign_cg",
    sequence_column="consensus_seq",
    # phred_column="consensus_phred",
    # query_start_column="query_start",
    # query_end_column="query_end",
    # query_length_column="query_length",
    # path_start_column="path_start",
    # path_end_column="path_end",
    struct_name="extract_segments2",
    keep_full=True,
)

In [ ]:
plt.hist(
    x.select(pl.col("consensus_seq").str.len_bytes())["consensus_seq"],
    bins=100,
    log=True,
);

In [ ]:
len(x[0, "consensus_seq"])

In [ ]:
x.select("cg", "realign_cg")

In [ ]:
y2[0, "extract_segments2"]

In [ ]:
y[0, "extract_segments2"]

In [ ]:
x["consensus_seq"]

In [ ]:
x["cg"].value_counts(sort=True)

In [ ]:
x[2, "cg"]

In [ ]:
x[2].select(r"^.*\|cigar$")

In [ ]:
df["promoter|variant"].value_counts(sort=True)

In [ ]:
df.filter(pl.col("promoter|variant") == "J23106", pl.col("promoter|divergence") == 0)[
    "promoter|seq"
].value_counts(sort=True).to_pandas()

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|seq"].value_counts(
    sort=True
).to_pandas()

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|cigar"].value_counts(
    sort=True
).to_pandas()

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|deletions"].value_counts(
    sort=True
)

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|divergence"].value_counts(
    sort=True
)

## Export to Eaton format

In [ ]:
%%time
df = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/references/pLIB476jqs.gfa"
)

In [ ]:
df = processing.compute_divergences(
    df, set([s.split("=")[0] for s in gfa.segment_names])
)

In [ ]:
df["max_divergence"].value_counts(sort=True)

In [ ]:
df.filter(pl.col("max_divergence") > 0.1)

In [ ]:
df.filter(
    pl.col("promoter_repressor|variant").is_not_null(),
)["max_divergence"]

In [ ]:
df["RBS_B0032m|matches"].value_counts()

In [ ]:
df["CDS_mVenus|matches"].value_counts(sort=True)

In [ ]:
df["CDS_mVenus|insertions"].value_counts(sort=True)

In [ ]:
df["CDS_mVenus|deletions"].value_counts(sort=True)

In [ ]:
df["CDS_mVenus|divergence"].value_counts(sort=True)

In [ ]:
df["promoter|variant"].value_counts()

In [ ]:
df["promoter_repressor|variant"].value_counts()

In [ ]:
x = df.filter(pl.col("promoter_repressor|variant") == "pLacI")[0]

In [ ]:
x[0, "promoter_repressor|seq"]

In [ ]:
df.filter(pl.col("promoter_repressor|variant") == "pPhlF")[0, "consensus_seq"]

In [ ]:
df.columns

In [ ]:
%%time
df_eaton = (
    df.filter(
        pl.col("primary_alignment"), pl.col("e2e"), pl.col("consensus_depth") >= 4
    )
    .with_columns(
        barcode=path_to_barcode_string("variants_path"),
        reference=pl.lit(""),
        alignmentstart=1,
        cigar=pl.lit(""),
        subsample=pl.lit(""),
    )
    .rename({"consensus_seq": "consensus"})
    .select(
        "barcode",
        "consensus",
        "reference",
        "alignmentstart",
        "cigar",
        "subsample",
        "promoter|variant",
        "SD2_variant|seq",
        "CDS_mVenus|divergence",
        "max_divergence",
    )
    .sort("barcode")
    .with_row_index(name="barcodeid")
    .with_row_index(name="")
)

In [ ]:
df_eaton

In [ ]:
df_eaton.write_csv("240703_pLIB476_bottlenecked_eaton_export.tsv", separator="\t")

In [ ]:
!pwd

# 240612_pLIB476_isolates

## prepare_reads

In [ ]:
%%time
arrow_filename = "/home/jqs1/scratch/sequencing/240612_pLIB476_isolates/output/vg/prepare_reads/*.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
%%time
df2 = df.with_columns(
    dup=pl.col("name").is_duplicated(),
    primary_alignment=pl.col("name").is_first_distinct(),
    e2e=pl.col("full_path")
    .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
    .list.len()
    == 2,
    bc_e2e=pl.col("full_path")
    .list.set_intersection(["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"])
    .list.len()
    == 2,
)

In [ ]:
df3 = (
    df2.filter(pl.col("bc_e2e"), pl.col("primary_alignment"))
    .unnest("extract_segments")
    .with_columns(
        sample=pl.col("name").str.split_exact("_", 2).struct[2].cast(pl.Int32)
    )
)

In [ ]:
df4 = (
    df3.group_by("sample")
    .agg(
        *[
            pl.col(f"BC:bit{bit}|{type_}").mean()
            for bit in range(30)
            for type_ in ("mismatches", "insertions", "deletions")
        ]
    )
    .sort("sample")
)

In [ ]:
df4

In [ ]:
df4.filter(pl.col("sample") == 1)

In [ ]:
len(mismatches)

In [ ]:
bits = np.arange(30)
for row in df4.to_dicts():
    mismatches = [row[f"BC:bit{bit}|mismatches"] for bit in bits]
    insertions = [row[f"BC:bit{bit}|insertions"] for bit in bits]
    deletions = [row[f"BC:bit{bit}|deletions"] for bit in bits]
    plt.figure(figsize=(10, 3))
    plt.stackplot(
        [-0.5, *(bits + 0.5)],
        [*mismatches, 0],
        [*insertions, 0],
        [*deletions, 0],
        labels=["mismatches", "insertions", "deletions"],
        step="post",
    )
    plt.ylim([0, 6])
    plt.title(f"sample {row['sample']}")
    plt.xticks(bits)
    plt.legend();

In [ ]:
sample = 4
bit = 17
bins = np.arange(10)
bin_centers = (bins[:-1] + bins[1:]) / 2
hists = {
    type_: np.histogram(
        df3.filter(pl.col("sample") == sample)[f"BC:bit{bit}|{type_}"], bins=bins
    )[0]
    for type_ in ("mismatches", "insertions", "deletions")
}
plt.figure(figsize=(6, 3))
plt.stackplot(
    bin_centers - 1,
    hists["mismatches"],
    hists["insertions"],
    hists["deletions"],
    labels=["mismatches", "insertions", "deletions"],
    step="post",
)
plt.title(f"sample {sample} bit {bit}")
plt.legend();

## extract_segments

In [ ]:
%%time
arrow_filename = "/home/jqs1/scratch/sequencing/240612_pLIB476_isolates/output/primary_max_divergence=1/extract_segments/*.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
df2 = (
    df.filter(pl.col("name").is_first_distinct())
    .with_columns(barcode_str=path_to_barcode_string("variants_path"))
    .sort("barcode_str")
    .to_pandas()
)

In [ ]:
df2[["barcode_str", "name", "grouping_depth"]]

In [ ]:
df2[["barcode_str", "name", "grouping_depth"]]

In [ ]:
df2.iloc[-1].loc["variants_path"]

In [ ]:
df2.iloc[-2].loc["variants_path"]

In [ ]:
%%time
arrow_filename = "/home/jqs1/scratch/sequencing/240612_pLIB476_isolates/output/primary_max_divergence=0.1/extract_segments/*.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
len(df.filter(pl.col("name").is_first_distinct()))

In [ ]:
df.filter(pl.col("name").is_first_distinct())["grouping_depth"].to_pandas()

In [ ]:
df2 = processing.compute_divergences(
    df, processing.unique_segments(df, "path"), struct_name="extract_segments"
)
df2 = df2.with_columns(
    sample=pl.col("name").str.split_exact("_", 2).struct[2].cast(pl.Int32)
)

In [ ]:
df2.filter(pl.col("sample") == 11).sort("max_divergence")

In [ ]:
plt.hist(df2.filter(pl.col("sample") == 11)["max_divergence"], bins=100);

In [ ]:
df.head(100)

## Closest distance histogram

In [ ]:
from sklearn.neighbors import KDTree

In [ ]:
%%time
arrow_filename = "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/output/max_divergence=0.05/extract_segments/*.arrow"
# arrow_filename = "/home/jqs1/scratch/sequencing/240612_pLIB476_isolates/output/primary_max_divergence=1/extract_segments/*.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
%%time
df2 = df.with_columns(
    dup=pl.col("name").is_duplicated(),
    is_primary_alignment=pl.col("name").is_first_distinct(),
    e2e=pl.col("variants_path")
    .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
    .list.len()
    == 2,
    bc_e2e=pl.col("variants_path")
    .list.set_intersection(["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"])
    .list.len()
    == 2,
)

In [ ]:
%%time
df3 = df2.filter(
    pl.col("is_primary_alignment"), pl.col("e2e"), pl.col("consensus_depth") >= 0
).with_columns(
    barcode2=pl.concat_list([f"BC:bit{idx}|variant" for idx in range(30)]).cast(
        pl.Array(pl.Boolean, 30)
    ),
    grouping_barcode=path_to_barcode_array("grouping_path"),
    variants_barcode=path_to_barcode_array("variants_path"),
)

In [ ]:
len(df3)

In [ ]:
grouping_barcodes = df3["grouping_barcode"].to_numpy().astype(np.int16)
variants_barcodes = df3["variants_barcode"].to_numpy().astype(np.int16)
barcodes2 = df3["barcode2"].to_numpy().astype(np.int16)

In [ ]:
(grouping_barcodes != variants_barcodes).any(axis=1).sum()

In [ ]:
grouping_barcodes[:10]

In [ ]:
variants_barcodes[:10]

In [ ]:
x = df3.filter(pl.col("barcode") == list(barcodes[2]))
x

In [ ]:
x[0, "grouping_path"].to_list()

In [ ]:
x[0, "variants_path"].to_list()

In [ ]:
x[0, "grouping_path"]

In [ ]:
df3.select(pl.col("barcode").is_duplicated().sum())

In [ ]:
names = df3["name"]

In [ ]:
names[9553]

In [ ]:
df3[1152, "barcode"].to_numpy()

In [ ]:
df3[9553, "barcode"].to_numpy()

In [ ]:
barcodes.shape

In [ ]:
print("\n".join(df3[1152, "variants_path"].to_list()))

In [ ]:
%%time
kd = KDTree(barcodes, leaf_size=40, metric="l1")
dists, closest = kd.query(barcodes, k=2)
dists = dists[:, 1].astype(np.int16)

In [ ]:
np.where(dists == 0)

In [ ]:
bc_dups = barcodes[dists == 0]

In [ ]:
(bc_dups * (2 ** np.arange(30))[np.newaxis, :]).sum(axis=1)

In [ ]:
counts = np.bincount(dists)
counts

In [ ]:
plt.step(np.arange(len(counts)), counts, where="mid");

In [ ]:
counts

In [ ]:
plt.step(np.arange(len(counts)), counts, where="mid");

# 240610_pLIB476 vs. 240510_pLIB473-476

In [ ]:
# arrow_filename = "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/"
arrow_filename = "/home/jqs1/scratch/sequencing/240513_pLIB473_476/20240513_1645_2C_PAW46239_b49d575f/prepare_reads.all_segments/*.arrow"
df = concat_glob(arrow_filename)  # .collect()

In [ ]:
gfa_filename = "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/references/pLIB476jqs.gfa"
gfa = gfapy.Gfa.from_file(gfa_filename)

In [ ]:
df.schema

In [ ]:
gfa.segment_names

In [ ]:
df2 = df.filter(pl.col("end_to_end")).head(100).collect()

In [ ]:
df2.with_columns(
    barcode=pl.concat_list(
        [
            pl.col("extract_segments").struct.field(f"BC:bit{idx}|variant")
            for idx in range(30)
        ]
    )
)["barcode"]

In [ ]:
df3[22, "full_path"].to_list()

In [ ]:
df.schema["extract_segments"].fields

In [ ]:
# TODO: use exclude after release including https://github.com/pola-rs/polars/issues/16661
seg_col = pl.col("extract_columns").struct.field
df2 = (
    df.head(10)
    .with_columns(
        divergence=pl.sum_horizontal(
            seg_col(r"\|(mismatches|insertions|deletions)").exclude(
                r"upstream\|(mismatches|insertions|deletions)",
                r"downstream\|(mismatches|insertions|deletions)",
            )
        )
    )
    .collect()
)

In [ ]:
seg_col = pl.col("extract_columns").struct.field
df2 = (
    df.head(10)
    .with_columns(
        divergence=pl.sum_horizontal(
            [
                seg_col(f"{s[1:]}|{type_}").fill_null(strategy="zero")
                for type_ in ("matches", "mismatches", "insertions", "deletions")
                for s in forward_segments
            ]
        )
    )
    .collect()
)

In [ ]:
df2["divergence"]

In [ ]:
df2 = df.filter(pl.col("end_to_end")).with_columns(
    barcode=pl.concat_list(
        [
            pl.col("extract_segments").struct.field(f"BC:bit{idx}|variant")
            for idx in range(30)
        ]
    )
)

In [ ]:
df3 = df2.head(100).collect()

In [ ]:
%%time
df4 = df2.filter(pl.col("barcode") == df3[22, "barcode"].to_list()).collect()

In [ ]:
df4